### 1.数据接口

In [1]:
# encoding: UTF-8
import logging
logger = logging.getLogger()

import requests
import json
import pandas as pd
import numpy as np

class SQI_DB(object):
    def __init__(self, url='http://182.254.241.204:5001', user='huangyangcheng@sqicapital.com', password='sq123456'):
        self.url = url
        self.user = user
        self.password = password
        self.token = self.generate_token()

    def generate_token(self):
        login_url = self.url + "/auth/login"
        headers = {'content-type': 'application/json'}
        login_info = {'email': self.user, 'password': self.password}
        login_response = requests.post(login_url, data=json.dumps(login_info), headers=headers)
        return json.loads(login_response.text)['auth_token']

    #-------------------------------------------------------------------------------------------------------------------
    #CCM日频时间序列接口
    def get_ssd(self, index_id, startday='', endday='',source='ifind', ret_type='df'):
        data_header = {'Authorization': 'Basic ' + self.token, 'content-type': 'application/json'}
        data_url = self.url + "/ssd"
        data_info = {
            'indexes': [
                {
                    'id': index_id,
                    'startday': startday,
                    'endday': endday,
                }
            ],
            'source':source
        }
        data_rsp = requests.post(data_url, data=json.dumps(data_info), headers=data_header)
        data = json.loads(json.loads(data_rsp.text)['data'])
        if ret_type == 'df':
            df = pd.DataFrame(data)
        else:
            # raise warning !
            raise ValueError('Unsupported ret_type={}'.format(ret_type))
        return df

    #CCM K线接口数据
    def get_bar(self, symbol, bartype, ret_type='df'):
        data_header = {'Authorization': 'Basic ' + self.token, 'content-type': 'application/json'}
        data_url = self.url + "/bar"
        data_info = {
            'index':
                {
                    'symbol': symbol,
                    'bartype': bartype    #'1 Min'  '5 Min' '15 Min'  '1 Hour' 'Daily'
                }
        }
        data_rsp = requests.post(data_url, data=json.dumps(data_info), headers=data_header)
        data = json.loads(json.loads(data_rsp.text)['data'])
        if ret_type == 'df':
            df = pd.DataFrame(data)
        else:
            raise ValueError('Unsupported ret_type={}'.format(ret_type))
        return df

### 2.生成 SQI_DB 对象

In [2]:
download = SQI_DB()

KeyError: 'auth_token'

### 3. CCM日频时间序列接口 get_ssd()
**交易品种:**
- **有色**：
1.铜(cu8888)
- **贵金属**：
2.黄金(AU8888)
- **黑色**：
3.铁矿(I8888), 4.螺纹钢(RB8888),
- **农产品**：
5.豆粕(M8888), 6.豆油(Y8888), 7.棕榈油(P8888), 8.棉花(CF8888)
- **化工**：
9.橡胶(RU8888), 10.原油(SC8888)

In [3]:
# ## 生成 "SQI_DB" 的对象
# id_list = ['ZN8888.SHF', 'CU8888.SHF', 'AG8888.SHF', 'AU8888.SHF', 'AL8888.SHF', 'I8888.DCE', 'RB8888.SHF',
#            'M8888.DCE', 'Y8888.DCE', 'P8888.DCE', 'CF8888.CZC',
#            'RU8888.SHF', 'SC8888.SHF', 'FU8888.SHF', 'SP8888.SHF',
#            'HC8888.SHF', 'NI8888.SHF', 'BU8888.SHF', 'SN8888.SHF',
#            'SA8888.CZC', 'TA8888.CZC', 'MA8888.CZC', 'OI8888.CZC',
#            'FG8888.CZC', 'V8888.DCE', 'EG8888.DCE', 'PP8888.DCE',
#            'PG8888.DCE', 'EB8888.DCE', 'L8888.DCE']
# df_list = []
# def append_df(target, list_id, list_df):
#     for i in range(len(list_id)):
#         df = target.get_ssd(list_id[i]).dropna()
#         df = df.drop(index=df[df['index_value'] == 0].index)
#         df['tradingday'] = df['tradingday'].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:8])
#         list_df.append(df)
# append_df(download, id_list, df_list)

In [7]:
# df_list[8]

In [8]:
# import datetime
# df = df_list[3]
# # df = dt.datetime.strptime(df_list[8]['tradingday'], '%Y-%m-%d')
# df['tradingday'] = pd.to_datetime(df['tradingday'], errors='coerce')
# df['month'] = df['tradingday'].apply(lambda x: x.month)
# df['first'] = df['tradingday'].where(df['month'] != df.shift(1)['month'], np.nan)
# df['last'] = df['tradingday'].where(df['month'] != df.shift(-1)['month'], np.nan)
# df = df[df['first'].notna() | df['last'].notna()]
# df = df.iloc[2:]
# df['tradingday'] = df['tradingday'].apply(lambda x: x.year)
#
# def ratio_to_peak(series):
#     return (series.iloc[-1]/series.iloc[0])-1
#
# def positive_probability(series):
#     return (series.sum()/len(series))*100
#
# print(datetime.datetime.now().year)
# # df = df.groupby(['tradingday', 'month'])["index_value"].agg(ratio_to_peak)
# # df = df.reset_index()
# # df['positive'] = df['index_value'].apply(lambda x: x>0).replace(True, 1).replace(False, 0)
# # df = df.groupby('month')[['index_value', 'positive']].agg(positive_probability)
# # month_avg = np.round(df['index_value'].to_list(),2)
# # month_possibility = np.round(df['positive'].to_list(),2)
# # display(df)
# # print(month_avg)
# # print(month_possibility)
# df[df['tradingday'] >= datetime.datetime.now().year-2]

In [12]:
import math
# def price_ratio(ar):
#     ratio_list = []
#     for i in range(len(ar)-1):
#         ratio_list.append(math.log((ar[i+1]/ar[i]), math.e))
#     return ratio_list

In [4]:
# category_list = ['有色', '有色', '贵金属', '贵金属', '有色', '黑色', '黑色',
#                  '农产品', '农产品', '农产品', '农产品',
#                  '化工', '化工', '化工', '化工',
#                  '黑色', '有色', '化工', '有色',
#                  '化工', '化工', '化工', '农产品',
#                  '化工', '化工', '化工', '化工',
#                  '化工', '化工', '化工']
# typo = 4
# cor_list = []
# for i in range(len(category_list)):
#     if category_list[i]  == '有色' and typo == 1:
#         cor_list.append((df_list[i].tail(55)['index_value'].to_list()))
#     elif category_list[i] == '贵金属' and typo == 2:
#         cor_list.append((df_list[i].tail(55)['index_value'].to_list()))
#     elif category_list[i] == '黑色' and typo == 3:
#         cor_list.append((df_list[i].tail(55)['index_value'].to_list()))
#     elif category_list[i] == '农产品' and typo == 4:
#         cor_list.append((df_list[i].tail(55)['index_value'].to_list()))
#     elif category_list[i] == '化工' and typo == 5:
#         cor_list.append((df_list[i].tail(55)['index_value'].to_list()))
# print(np.corrcoef(cor_list))
# # print(np.corrcoef(df_list[8].tail(55)['index_value'].to_list(), df_list[9].tail(55)['index_value'].to_list()))
# # print(len(np.corrcoef(cor [0.97955664 1.        ]]_list[0],cor_list[1])))

### 4.CCM K线接口数据 get_bar()

In [12]:
bar_Cu = download.get_bar('cu2305', 'Daily')
bar_Cu

,tradingday,bartime,openprice,highprice,lowprice,closeprice,volume,openinterest
0,20220517,21:01,70330.0,70330.0,70330.0,70330.0,0,0.0
1,20220518,21:01,70770.0,70770.0,70480.0,70710.0,23,21.0
2,20220519,21:01,70220.0,70710.0,70220.0,70300.0,3,22.0
3,20220520,21:01,70740.0,71000.0,70300.0,71000.0,7,27.0
4,20220523,21:01,71100.0,71100.0,70900.0,70900.0,12,33.0
...,...,...,...,...,...,...,...,...
219,20230407,21:01,68400.0,69090.0,68120.0,69040.0,106360,165833.0
220,20230410,21:01,68810.0,69050.0,68620.0,68740.0,58300,162497.0
221,20230411,21:01,68280.0,69040.0,68280.0,68880.0,94544,168974.0
222,20230412,21:01,68880.0,69310.0,68740.0,68860.0,106228,167373.0


### 5. Bubble Distribution
**standard deviation**: $s=\sqrt{\frac{(\sum{X-\bar{x})^2}}{N}}$

In [22]:
# ## 分位数 -> [0,100,5]
# import math
# def price_quantile(ar):
#     price = ar[len(ar)-1]
#     ar.sort()
#     l = len(ar)
#     for i in range(l):
#         if ar[i]>price:
#             return int((i/l)*100)
#         elif i == l-1:
#             return 100
#
#
# ## 波动率 -> list
# ## 取55天算std
# def volatility(ar):
#     vol_list = []
#     vol_std_list = []
#     for i in range(len(ar)-1):
#         vol_list.append(math.log((ar[i+1]/ar[i]), math.e))
#     for i in range(len(ar)-54):
#         vol_std_list.append(np.std(vol_list[i:i+55])*15.5)
#     return vol_std_list
#
# def price_ratio(ar):
#     ratio_list = []
#     for i in range(len(ar)-1):
#         ratio_list.append(math.log((ar[i+1]/ar[i]), math.e))
#     return ratio_list

### 可视化

In [9]:
# ### Scatter Plot
# from pyecharts.globals import CurrentConfig, NotebookType
# CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
# import pyecharts.options as opts
# from pyecharts.charts import Scatter
# from pyecharts.commons.utils import JsCode
# from pyecharts.faker import Faker
# bubble_point = []
# label_list = ['铜CU8888.SHF', '黄金AU8888.SHF', '铁矿I8888.DCE', '螺纹钢RB8888.SHF', '豆粕M8888.DCE','豆油Y8888.DCE','棕榈油P8888.DCE',
#                  '棉花CF8888.CZC','橡胶RU8888.SHF','原油SC8888.SHF']
# for i in range(len(df_list)):
#     arr1 = df_list[i]['index_value'].to_list()
#     arr2 = df_list[i]['index_value'].to_list()
#     bubble_point.append([price_quantile(arr1), price_quantile(volatility(arr2))])
# x_data = [d[0] for d in bubble_point]
# y_data = [d[1] for d in bubble_point]
# print(x_data)
# print(y_data)
# print(y_data[0])
# scatter = (
#     Scatter(init_opts=opts.InitOpts(bg_color='rgb(255,255,255)'))
#     .add_xaxis(xaxis_data = x_data[0:1])
#     .add_yaxis(series_name='有色',y_axis = y_data[0:1], label_opts=opts.LabelOpts(position = 'right',formatter='铜CU8888.SHF'), symbol_size=40)
#     .add_xaxis(xaxis_data = x_data[1:2])
#     .add_yaxis(series_name='贵金属',y_axis = y_data[1:2], label_opts=opts.LabelOpts(position = 'right',formatter='黄金AU8888.SHF'), symbol_size=40)
#     .add_xaxis(xaxis_data = x_data[2:3])
#     .add_yaxis(series_name='黑色',y_axis = y_data[2:3], label_opts=opts.LabelOpts(position = 'right',formatter='铁矿I8888.DCE'), symbol_size=40)
#     .add_xaxis(xaxis_data = x_data[3:4])
#     .add_yaxis(series_name='黑色',y_axis = y_data[3:4], label_opts=opts.LabelOpts(position = 'right',formatter='螺纹钢RB8888.SHF'), symbol_size=40)
#     .add_xaxis(xaxis_data = x_data[4:5])
#     .add_yaxis(series_name='农产品',y_axis = y_data[4:5], label_opts=opts.LabelOpts(position = 'right',formatter='豆粕M8888.DCE'), symbol_size=40)
#     .add_xaxis(xaxis_data = x_data[5:6])
#     .add_yaxis(series_name='农产品',y_axis = y_data[5:6], label_opts=opts.LabelOpts(position = 'right',formatter='豆油Y8888.DCE'), symbol_size=40)
#     .add_xaxis(xaxis_data = x_data[6:7])
#     .add_yaxis(series_name='农产品',y_axis = y_data[6:7], label_opts=opts.LabelOpts(position = 'right',formatter='棕榈油P8888.DCE'), symbol_size=40)
#     .add_xaxis(xaxis_data = x_data[7:8])
#     .add_yaxis(series_name='农产品',y_axis = y_data[7:8], label_opts=opts.LabelOpts(position = 'right',formatter='棉花CF8888.CZC'), symbol_size=40)
#     .add_xaxis(xaxis_data = x_data[8:9])
#     .add_yaxis(series_name='化工',y_axis = y_data[8:9], label_opts=opts.LabelOpts(position = 'right',formatter='橡胶RU8888.SHF'), symbol_size=40)
#     .add_xaxis(xaxis_data = x_data[9:10])
#     .add_yaxis(series_name='化工',y_axis = y_data[9:10], label_opts=opts.LabelOpts(position = 'right',formatter='原油SC8888.SHF'), symbol_size=40)
#     .set_series_opts()
#     .set_global_opts(
#         xaxis_opts=opts.AxisOpts(
#             name = '价格分位数(%)',
#             name_location= 'middle', name_gap= 40,
#             type_="value", splitline_opts=opts.SplitLineOpts(is_show=True)
#         ),
#         yaxis_opts=opts.AxisOpts(
#             type_="value",
#             name = '波动率分位数(%)',
#             name_location= 'middle', name_gap= 40,
#         ),
#         tooltip_opts=opts.TooltipOpts(
#             formatter=JsCode(
#                 "function (params) {return '价格分位数:' + params.value[0] + '%' + '<br>' + '波动率分位数:' + params.value[1] + '%';}"
#             )
#         ),
#         title_opts=opts.TitleOpts(title="期货交易 (价格 vs. 波动率分位数)"),
#     )
# )
# scatter.render("basic_scatter_chart.html")
# scatter.load_javascript()